## Análise de qualidade dos dados

Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np
import pyodbc

Funções do projeto:
- load_csv
- load_sql_table
- normalize_columns
- check_individual_quality
- check_data_quality

In [6]:
def load_csv(csv_path):
    """Carrega o arquivo CSV em um DataFrame, garantindo a correta separação de colunas."""
    return pd.read_csv(csv_path, delimiter=";") 

def load_sql_table(query, connection_string):
    """Carrega a tabela do SQL Server em um DataFrame."""
    conn = pyodbc.connect(connection_string)
    df = pd.read_sql(query, conn)
    conn.close()
    return df

def normalize_columns(df):
    """Padroniza os nomes das colunas removendo espaços e convertendo para minúsculas."""
    df.columns = df.columns.str.strip().str.lower()
    return df

def check_individual_quality(df, name="Tabela"):
    """
    Gera linhas de relatório sobre a qualidade de dados de uma única fonte:
      - Quantidade de duplicados
      - Quantidade de valores nulos
      - Estatísticas descritivas para colunas numéricas
    Retorna uma lista de strings em formato Markdown.
    """
    lines = []
    lines.append(f"### Checagem de Qualidade - {name}")

    # 1. Duplicados
    dup_count = df.duplicated().sum()
    lines.append(f"- **Duplicadas**: {dup_count}")

    # 2. Valores Nulos
    null_count = df.isnull().sum().sum()  # Soma total de nulos
    lines.append(f"- **Total de valores nulos**: {null_count}")

    # 3. Estatísticas descritivas (colunas numéricas)
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        lines.append("")
        lines.append("#### 📈 Estatísticas Descritivas (Colunas Numéricas)")
        lines.append("| Coluna | Mínimo | Máximo | Média | Mediana | Desvio Padrão |")
        lines.append("|--------|--------|--------|-------|---------|---------------|")
        for col in sorted(numeric_cols):
            col_min = round(df[col].min(), 2) if df[col].count() > 0 else "NaN"
            col_max = round(df[col].max(), 2) if df[col].count() > 0 else "NaN"
            col_mean = round(df[col].mean(), 2) if df[col].count() > 0 else "NaN"
            col_median = round(df[col].median(), 2) if df[col].count() > 0 else "NaN"
            col_std = round(df[col].std(), 2) if df[col].count() > 1 else "NaN"
            lines.append(f"| {col} | {col_min} | {col_max} | {col_mean} | {col_median} | {col_std} |")
    else:
        lines.append("")
        lines.append("Não há colunas numéricas para análise estatística.")
    
    lines.append("")
    return lines

def check_data_quality(csv_df, sql_df):
    """
    Gera um relatório Markdown com:
      - Sessão de checagem individual das tabelas (CSV e SQL)
      - Sessão de comparativos (número de linhas, colunas, tipos de dados, somas, valores faltantes, etc.)
    """
    # Normalizar os nomes das colunas para comparação correta
    csv_df = normalize_columns(csv_df)
    sql_df = normalize_columns(sql_df)

    
    # Início do relatório
    report_lines = []
    report_lines.append("# 📄 Relatório de Qualidade de Dados")
    report_lines.append("")

    # 1. Sessão de Checagem Individual das Tabelas
    report_lines.append("## 🔎 Checagem Individual das Tabelas")

    # Checagem do CSV
    report_lines.extend(check_individual_quality(csv_df, name="CSV"))

    # Checagem do SQL
    report_lines.extend(check_individual_quality(sql_df, name="SQL"))

    # 2. Sessão de Comparativos
    report_lines.append("## ⚖️ Comparação de Dados")

    # 2.1 Overview de linhas
    report_lines.append("### 📋 Overview")
    report_lines.append(f"- **Número de linhas no CSV**: **{len(csv_df)}**")
    report_lines.append(f"- **Número de linhas na tabela SQL**: **{len(sql_df)}**")
    report_lines.append("")

    # 2.2 Comparação de colunas
    report_lines.append("### 🔠 Comparação de Colunas")
    csv_columns = set(csv_df.columns)
    sql_columns = set(sql_df.columns)
    common_columns = csv_columns.intersection(sql_columns)
    missing_in_sql = csv_columns - sql_columns
    missing_in_csv = sql_columns - csv_columns

    report_lines.append("#### Colunas em comum:")
    if common_columns:
        report_lines.append(", ".join(sorted(common_columns)))
    else:
        report_lines.append("Nenhuma coluna em comum.")
    report_lines.append("")

    if missing_in_sql:
        report_lines.append("#### Colunas ausentes na tabela SQL:")
        report_lines.append(", ".join(sorted(missing_in_sql)))
        report_lines.append("")

    if missing_in_csv:
        report_lines.append("#### Colunas ausentes no CSV:")
        report_lines.append(", ".join(sorted(missing_in_csv)))
        report_lines.append("")
    else:
        report_lines.append("Não há colunas ausentes no CSV.")
    report_lines.append("")

    # 2.3 Comparação de Tipos de Dados
    report_lines.append("### 🔄 Comparação de Tipos de Dados")
    report_lines.append("| Coluna | Tipo no CSV | Tipo no SQL |")
    report_lines.append("|--------|-------------|-------------|")
    for col in sorted(common_columns):
        csv_type = csv_df[col].dtype
        sql_type = sql_df[col].dtype
        report_lines.append(f"| {col} | {csv_type} | {sql_type} |")
    report_lines.append("")

    # 2.4 Comparação de somas para colunas numéricas
    report_lines.append("### 💰 Comparação de Somatórias (Colunas Numéricas)")
    report_lines.append("| Coluna | Soma no CSV | Soma no SQL | Diferença |")
    report_lines.append("|--------|-------------|-------------|-----------|")
    numeric_cols = csv_df.select_dtypes(include=[np.number]).columns.intersection(common_columns)
    for col in sorted(numeric_cols):
        sum_csv = round(csv_df[col].sum(), 2)
        sum_sql = round(sql_df[col].sum(), 2)
        diff = round(sum_csv - sum_sql, 2)
        report_lines.append(f"| {col} | {sum_csv} | {sum_sql} | {diff} |")
    report_lines.append("")

    # 2.5 Valores faltantes por coluna
    report_lines.append("### 🕳️ Valores Faltantes (Comparativo)")
    report_lines.append("| Coluna | Faltantes no CSV | Faltantes no SQL |")
    report_lines.append("|--------|------------------|------------------|")
    for col in sorted(common_columns):
        missing_csv = csv_df[col].isna().sum()
        missing_sql = sql_df[col].isna().sum()
        report_lines.append(f"| {col} | {missing_csv} | {missing_sql} |")
    report_lines.append("")

    # 2.6 Estatísticas descritivas (comparativo) para colunas numéricas
    report_lines.append("### 📊 Estatísticas Descritivas (Comparativo)")
    if numeric_cols.any():
        report_lines.append("| Coluna | Média CSV | Média SQL | Desvio Padrão CSV | Desvio Padrão SQL |")
        report_lines.append("|--------|----------|-----------|-------------------|-------------------|")
        for col in sorted(numeric_cols):
            mean_csv = round(csv_df[col].mean(), 2)
            mean_sql = round(sql_df[col].mean(), 2)
            std_csv = round(csv_df[col].std(), 2) if csv_df[col].count() > 1 else "NaN"
            std_sql = round(sql_df[col].std(), 2) if sql_df[col].count() > 1 else "NaN"
            report_lines.append(f"| {col} | {mean_csv} | {mean_sql} | {std_csv} | {std_sql} |")
    else:
        report_lines.append("Não há colunas numéricas disponíveis para estatísticas descritivas.")
    report_lines.append("")

    # 2.7 Resumo de Erros
    report_lines.append("### ⚠️ Resumo de Erros")
    errors = []
    if len(csv_df) != len(sql_df):
        errors.append(f"Diferença no número de linhas: CSV possui {len(csv_df)} linhas, enquanto SQL possui {len(sql_df)}.")
    if missing_in_csv or missing_in_sql:
        errors.append("Diferenças nas colunas presentes entre CSV e SQL.")
    for col in numeric_cols:
        sum_csv = round(csv_df[col].sum(), 2)
        sum_sql = round(sql_df[col].sum(), 2)
        if sum_csv != sum_sql:
            errors.append(f"Diferença na somatória da coluna **{col}**: CSV = {sum_csv}, SQL = {sum_sql}.")
    if errors:
        for err in errors:
            report_lines.append(f"- {err}")
    else:
        report_lines.append("Nenhum erro detectado.")
    report_lines.append("")

    return "\n".join(report_lines)

Definição das configurações, execução da função e armazenamento do markdown

In [ ]:
if __name__ == "__main__":
    # Configurações
    csv_path = r"C:\Users\eunat\Desktop\Bank_Churn_Messy@.csv"
    connection_string = (
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=DESKTOP-QL6I1JD;"
        "DATABASE=Bank_Churn_Quality;"
        "Trusted_Connection=yes;"
    )

    query = "SELECT * FROM Customers"

    # Carregar os dados
    print("Carregando CSV...")
    csv_df = load_csv(csv_path)
    print("Carregando dados do SQL Server...")
    sql_df = load_sql_table(query, connection_string)

    # Gerar o relatório de qualidade
    print("Executando verificações de qualidade...")
    report_md = check_data_quality(csv_df, sql_df)

    # Salvar o relatório em um arquivo Markdown
    report_file = r"C:\Users\eunat\Desktop\data_quality_report.md"
    with open(report_file, "w", encoding="utf-8") as f:
        f.write(report_md)

    print(f"Relatório de qualidade gerado com sucesso em '{report_file}'.")